In [3]:
import pandas as pd
melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')


In [29]:
melb_df = melb_data.copy()

In [3]:
melb_df = melb_df.drop(['index', 'Coordinates'], axis=1)
melb_df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067,...,1,1,202.0,126.0,1970,Yarra,-37.7996,144.9984,Northern Metropolitan,4019
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067,...,1,0,156.0,79.0,1900,Yarra,-37.8079,144.9934,Northern Metropolitan,4019
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067,...,2,0,134.0,150.0,1900,Yarra,-37.8093,144.9944,Northern Metropolitan,4019
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067,...,2,1,94.0,126.0,1970,Yarra,-37.7969,144.9969,Northern Metropolitan,4019
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067,...,1,2,120.0,142.0,2014,Yarra,-37.8072,144.9941,Northern Metropolitan,4019


In [4]:
total_rooms = melb_df['Rooms'] + melb_df['Bedroom'] + melb_df['Bathroom']
display(total_rooms)

0         5
1         5
2         8
3         8
4         8
         ..
13575    10
13576     8
13577     8
13578     9
13579     9
Length: 13580, dtype: int64

In [5]:
melb_df['MeanRoomsSquare'] = melb_df['BuildingArea'] / total_rooms
display(melb_df['MeanRoomsSquare'])

0        25.200000
1        15.800000
2        18.750000
3        15.750000
4        17.750000
           ...    
13575    12.600000
13576    16.625000
13577    15.750000
13578    17.444444
13579    12.444444
Name: MeanRoomsSquare, Length: 13580, dtype: float64

In [6]:
diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area
display(melb_df['AreaRatio'])

0       -0.231707
1       -0.327660
2        0.056338
3        0.145455
4        0.083969
           ...   
13575   -0.676093
13576   -0.429185
13577   -0.551601
13578   -0.693060
13579   -0.527426
Name: AreaRatio, Length: 13580, dtype: float64

In [7]:
countries_df = pd.DataFrame({
    'country': ['Англия', 'Канада', 'США', 'Россия', 'Украина', 'Беларусь', 'Казахстан'],
    'population': [56.29, 38.05, 322.28, 146.24, 45.5, 9.5, 17.04],
    'square': [133396, 9984670, 9826630, 17125191, 603628, 207600, 2724902]
})
display(countries_df['population'])
countries_df['density'] = countries_df['population'] / countries_df['square'] * 1e6
print(round(countries_df['density'].mean(), 2))

0     56.29
1     38.05
2    322.28
3    146.24
4     45.50
5      9.50
6     17.04
Name: population, dtype: float64

84.93


# Задание 3.3
## Создайте в таблице melb_df признак WeekdaySale (день недели). Найдите, сколько объектов недвижимости было продано в выходные (суббота и воскресенье), результат занесите в переменную weekend_count. В качестве ответа введите результат вывода переменной weekend_count.

In [6]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
melb_df['Date'] = pd.to_datetime(melb_df['Date'])
melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek

weekend_count = melb_df[(melb_df['WeekdaySale'] == 5) | (melb_df['WeekdaySale'] == 6)]['WeekdaySale'].count()
weekend_count


12822

# Задание 4.2
##### Ранее, в задании 3.3, мы создали признак WeekdaySale в таблице melb_df — день недели продажи. Из полученных в задании результатов можно сделать вывод, что объекты недвижимости в Мельбурне продаются преимущественно по выходным (суббота и воскресенье).
##### Напишите функцию get_weekend(weekday), которая принимает на вход элемент столбца WeekdaySale и возвращает 1, если день является выходным, и 0 — в противном случае, и создайте столбец Weekend в таблице melb_df с помощью неё.

##### Примените эту функцию к столбцу и вычислите среднюю цену объекта недвижимости, проданного в выходные дни. Результат округлите до целых.

In [7]:
def get_weekend(weekday):
    return [0, 1][weekday in [5 ,6]]

melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)

melb_df[(melb_df['Weekend'] == 1)]['Price'].mean()


1081198.6406956792

# Задание 4.3
#### Преобразуйте столбец SellerG с наименованиями риелторских компаний в таблице melb_df следующим образом: оставьте в столбце только 49 самых популярных компаний, а остальные обозначьте как 'other'.Найдите, во сколько раз минимальная цена объектов недвижимости, проданных компанией 'Nelson', больше минимальной цены объектов, проданных компаниями, обозначенными как 'other'. Ответ округлите до десятых

In [33]:
popular_company = melb_df['SellerG'].value_counts().nlargest(49)

melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in popular_company else 'other')
#melb_df[(melb_df['SellerG'] == 'other')].head(50)
melb_df[(melb_df['SellerG'] == 'Nelson')]['Price'].min() / melb_df[(melb_df['SellerG'] == 'other')]['Price'].min()

1.297709923664122

# ЗАДАНИЕ 4.4
#### Представьте, что вы занимаетесь подготовкой данных о вакансиях с платформы hh.ru. В вашем распоряжении имеется таблица, в которой с помощью парсинга собраны резюме кандидатов. В этой таблице есть текстовый столбец «Опыт работы». Пример такого столбца представлен ниже в виде объекта Series. Структура текста в столбце фиксирована и не может измениться.

#### Напишите функцию get_experience(arg), аргументом которой является строка столбца с опытом работы. Функция должна возвращать опыт работы в месяцах. Не забудьте привести результат к целому числу.

#### Обратите внимание, что опыт работы может выражаться только в годах или только в месяцах. Учтите это при построении своей функции. Кроме того, учтите возможные вариации слов месяц (месяца, месяцев) и год (года, лет).

#### Примените вашу функцию к Series experience_col с помощью метода apply().



In [67]:
from calendar import month
import re
experience_col = pd.Series([
        'Опыт работы 8 лет 3 месяца',
        'Опыт работы 3 года 5 месяцев',
        'Опыт работы 1 год 9 месяцев',
        'Опыт работы 3 месяца',
        'Опыт работы 6 лет'
        ])
def get_experience(arg):
    """
    Напишите функцию get_experience(arg), аргументом которой является строка столбца с опытом работы. 
    Функция должна возвращать опыт работы в месяцах. Не забудьте привести результат к целому числу.
    """
    #res = re.findall(r'\d\s\S{0,10}', arg)
    months = 0
    for dt in re.findall(r'\d{1,3}\s\S{0,10}', arg):
        val = int(re.findall(r'\d{1,3}', dt)[0])
        months += [val, val*12][dt.find('м') == -1]
    
    return months

print(get_experience('12 месяцев'))

12 месяцев
12


# Category type

In [77]:
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
display(melb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   index          13580 non-null  int64   
 1   Suburb         13580 non-null  object  
 2   Address        13580 non-null  object  
 3   Rooms          13580 non-null  int64   
 4   Type           13580 non-null  category
 5   Price          13580 non-null  float64 
 6   Method         13580 non-null  category
 7   SellerG        13580 non-null  category
 8   Date           13580 non-null  object  
 9   Distance       13580 non-null  float64 
 10  Postcode       13580 non-null  int64   
 11  Bedroom        13580 non-null  int64   
 12  Bathroom       13580 non-null  int64   
 13  Car            13580 non-null  int64   
 14  Landsize       13580 non-null  float64 
 15  BuildingArea   13580 non-null  float64 
 16  YearBuilt      13580 non-null  category
 17  CouncilArea    12211 non-null  

None

In [86]:
melb_df['Type'] = melb_df['Type'].cat.rename_categories({
    'u': 'unit',
    't': 'townhouse',
    'h': 'house'
})
display(melb_df['Type'])
display(melb_df['Type'])

0        house
1        house
2        house
3        house
4        house
         ...  
13575    house
13576    house
13577    house
13578    house
13579    house
Name: Type, Length: 13580, dtype: category
Categories (3, object): ['house', 'townhouse', 'unit']

0        house
1        house
2        house
3        house
4        house
         ...  
13575    house
13576    house
13577    house
13578    house
13579    house
Name: Type, Length: 13580, dtype: category
Categories (3, object): ['house', 'townhouse', 'unit']

In [87]:
melb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   index          13580 non-null  int64   
 1   Suburb         13580 non-null  object  
 2   Address        13580 non-null  object  
 3   Rooms          13580 non-null  int64   
 4   Type           13580 non-null  category
 5   Price          13580 non-null  float64 
 6   Method         13580 non-null  category
 7   SellerG        13580 non-null  category
 8   Date           13580 non-null  object  
 9   Distance       13580 non-null  float64 
 10  Postcode       13580 non-null  int64   
 11  Bedroom        13580 non-null  int64   
 12  Bathroom       13580 non-null  int64   
 13  Car            13580 non-null  int64   
 14  Landsize       13580 non-null  float64 
 15  BuildingArea   13580 non-null  float64 
 16  YearBuilt      13580 non-null  category
 17  CouncilArea    12211 non-null  

# Задание 5.2
#### При преобразовании столбцов таблицы о недвижимости к типу category мы оставили без внимания столбец Suburb (пригород). Давайте исправим это. С помощью метода info() узнайте, сколько памяти занимает таблица melb_df. Преобразуйте признак Suburb следующим образом: оставьте в столбце только 119 наиболее популярных пригородов, остальные замените на 'other'. Приведите данные в столбце Suburb к категориальному типу. В качестве ответа запишите разницу между объёмом занимаемой памяти до преобразования (который мы получили ранее в модуле) и после него в Мб. Ответ округлите до десятых.

In [95]:
pd.set_option('display.max_columns', 40)
pd.set_option('display.max_rows', 40)

In [98]:
suburb = melb_df['Suburb'].value_counts().nlargest(119)
melb_df['Suburb'] = melb_df['Suburb'].apply(lambda x: x if x in suburb else 'other')

other              2215
Reservoir           359
Richmond            260
Bentleigh East      249
Preston             239
                   ... 
Greensborough        36
Caulfield North      35
Chadstone            35
Hughesdale           35
Mont Albert          34
Name: Suburb, Length: 119, dtype: int64

In [100]:
melb_df['Suburb'] = melb_df['Suburb'].astype('category')
melb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   index          13580 non-null  int64   
 1   Suburb         13580 non-null  category
 2   Address        13580 non-null  object  
 3   Rooms          13580 non-null  int64   
 4   Type           13580 non-null  category
 5   Price          13580 non-null  float64 
 6   Method         13580 non-null  category
 7   SellerG        13580 non-null  category
 8   Date           13580 non-null  object  
 9   Distance       13580 non-null  float64 
 10  Postcode       13580 non-null  int64   
 11  Bedroom        13580 non-null  int64   
 12  Bathroom       13580 non-null  int64   
 13  Car            13580 non-null  int64   
 14  Landsize       13580 non-null  float64 
 15  BuildingArea   13580 non-null  float64 
 16  YearBuilt      13580 non-null  category
 17  CouncilArea    12211 non-null  